# <p style="text-align: center;"> Data Collection</p>

## Yelp data retrieval
In this notebook, we will be using Python, Yelp API, and a third party API (scraperdog), to scrape data from yelp.  All of the data scraped is then saved into a CSV file to analyze later on. Our scraper will scrape each restaurant for 9 pages and grab 10 reviews per page. 

**NOTE:** API keys have been removed for privacy purposes, in order to run this, you will need to have your own key.

In [4]:
import pandas as pd
from yelpapi import YelpAPI
from bs4 import BeautifulSoup
import requests
import csv

client_id = " "
api_key = " "

yelp_api = YelpAPI(api_key)

#scraping restaurants from salt lake city, UT
term = 'Restaurant'
location = 'Salt Lake City, UT'
search_limit = 50
response = yelp_api.search_query(term = term,
                                 location = location,
                                 limit = search_limit)
#print(response)

In [1]:
scrapingdog_API_key = ' '
yelp_url = 'https://www.yelp.com/biz/hallpass-salt-lake-city'
page_limit = 9

#Finding the page url and pages that we want to scrape for each restaurant
def scrape_page(url, page, data_list):
    scrape_url = url + '?start=' + str(page*10)
    scrapingdog_url = 'https://api.scrapingdog.com/scrape?api_key=' + scrapingdog_API_key + '&url='
    combine_url = scrapingdog_url + scrape_url + '&dynamic=True'
    r = requests.get(combine_url).text
    soup = BeautifulSoup(r, 'html.parser')
    list_class = 'review__373c0__13kpL'
    review_list = soup.find_all('div', {'class' : list_class})
    print('scraping :', combine_url)
    #loop over review list and adding to data_list
    for l in review_list:
        data = parse_review(l)
        data_list.append(data)
    #if the page is less than 10 and the length of the page is a full page then continue to the next page
    if page < page_limit and len(review_list) == 10:
        scrape_page(url, page+1, data_list)
    else:
        return data_list

#Finding data that we might want for later, but mostly the reviews and reviewer ratings
def parse_review(review):
    content = review.find('p', {'class' : 'comment__373c0__1M-px css-n6i4z7'}).text
    reviewer_rating = review.find('div', {'class' : 'i-stars__373c0__1T6rz'}).attrs['aria-label'].split()[0]
    reviewer_date = review.find('span', {'class' : 'css-e81eai'}).text
    reviewers_name = review.find('span',{'class':'css-m6anxm'}).text
    reviewer_tags = review.find_all('button', {'class' : 'button__373c0__2Q-29'})
    useful_tag = reviewer_tags[0].text[6:].strip()
    if not useful_tag:
        useful_tag = 0
        
    funny_tag = reviewer_tags[1].text[5:].strip()
    if not funny_tag:
        funny_tag = 0
    
    cool_tag = reviewer_tags[2].text[4:].strip()
    if not cool_tag:
        cool_tag = 0
    
    data = {'content' : content, 'review_rating' : reviewer_rating, 'date' : reviewer_date, 'reviewers_name' : reviewers_name, 
           'cool_tag' : cool_tag, 'useful_tag' : useful_tag, 'funny_tag' : funny_tag}    
    return data

#putting it all together into a CSV to analyze
def scrape_business(business, is_first):
    print('scraping business: ' + business['name'])
    data_list = []
    business_keys = ['name','categories','rating','coordinates','transactions','price', 'is_closed']
    scrape_page(business['url'], 0, data_list)

    #creating a csv of our reviews
    with open('reviews.csv', 'a', encoding="utf8", newline='') as file: 
        review_keys = ['content', 'review_rating', 'date', 'reviewers_name', 'cool_tag','useful_tag','funny_tag']
        
        #combining headers for both sets of data
        dict_writer = csv.DictWriter(file, business_keys + review_keys)
        
        #writing out each individual CSV row
        #creating the header for the first business only
        if is_first:
            dict_writer.writeheader()
        for row in data_list:
            for key in business_keys:
                row[key] = business[key] if key in business else ""
            dict_writer.writerow(row)

In [1]:
# #finding business info and calling function
for index in range(len(response['businesses'])):
    scrape_business(response['businesses'][index], index==0) #whether or not this is the first restaurant in the file